## 9주차 recsys basic 과제: Q1, Q2에있는 빈칸을 채워주세요

## Movielens data 탐방하기

In [1]:

import argparse

import torch

#다양한 hyperparameter를 조절하기 위한 argparser
argparser = argparse.ArgumentParser()
argparser.add_argument('--latent_dim', type=int, default=10)
argparser.add_argument('--batch_size', type=int, default=500)
args = argparser.parse_args("")

In [3]:
import pandas as pd
import numpy as np
# 이건 토치용입니다.
from torch.utils.data import Dataset, DataLoader
import torch

class MlDataLoader(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    

## 무비렌즈 데이터 불러주는  class: ml100k
ML100k 클래스에서 data_path만 여러분이 u.data를 불러올 수 있는 경로로 바꿔주세요!!

In [8]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 37.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 64.7 MB/s eta 0:00:0000:0100:01


In [11]:
import pandas as pd
# traintest split
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
#from src.preprocess.mldataloader import MlDataLoader


class ML100k():

    def __init__(self,args) -> None:
        self.args=args
        self.call_data(args)
        pass

    def call_data(self, args) -> None:
        self.args = args
        data_path="../u.data" #이부분을 다운받은 데이터를 코랩에서 실행할 수 잇도록 바꿔주세요

        self.data= pd.read_csv(data_path, sep='\t', header=None, names=['user', 'item', 'rating', 'timestamp'])
        self.num_users = self.data.user.nunique()
        self.num_items = self.data.item.nunique()
        self.num_ratings = len(self.data)    

        # user, item index를 0부터 시작하도록 변경-. nn.Embedding에서 index가 0부터 시작하므로 1을 빼줍니다.
        self.data.user = self.data.user -1
        self.data.item = self.data.item -1
        
        
        self.x = self.data[['user', 'item']].values
        self.y= self.data.rating.values

        # split data into train and test
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.x, self.y, test_size=0.2, random_state=42)
        
        
    

    def get_dataframe(self):
        # sort dataframe by user
        return self.data[['user', 'item','rating']]
    
        
    def load_torch_data(self):

        # convert x to integer, y to floattenser
        self.x_train = torch.tensor(self.x_train, dtype=torch.long)
        self.y_train = torch.tensor(self.y_train, dtype=torch.float)
        self.x_test = torch.tensor(self.x_test, dtype=torch.long)
        self.y_test = torch.tensor(self.y_test, dtype=torch.float)
        

        train_dataset = MlDataLoader(self.x_train, self.y_train)
        test_dataset = MlDataLoader(self.x_test, self.y_test)

        train_loader = DataLoader(train_dataset, batch_size=self.args.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=self.args.batch_size, shuffle=False)

        return train_loader, test_loader


In [12]:
ml100k=ML100k(args)
dataframe=ml100k.get_dataframe()

In [13]:
dataframe# 아래와 같이 user, item, rating으로 이루어진 데이터프레임이 나오게 됩니다. 

,user,item,rating
0,195,241,3
1,185,301,3
2,21,376,1
3,243,50,2
4,165,345,1
...,...,...,...
99995,879,475,3
99996,715,203,5
99997,275,1089,1
99998,12,224,2


##  Q1 : 해당 데이터 프레임을 통해서 각각의 유저가 item들의 점수를 어떻게 매겼는지를 나타내는 user-item matrix(rating matrix)를 만들어 주세요!!
이 때 유저가 시청하지 않은 데이터는 -1로 처리해주세요!

In [14]:
# 여기에 코드를 작성해주세요
rating_matrix=dataframe.pivot_table(index='user',columns='item',values='rating',fill_value=-1)
rating_matrix

item,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
user,,,,,,,,,,,,,,,,,,,,,
0,5,3,4,3,3,5,4,1,5,3,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,4,-1,-1,-1,-1,-1,-1,-1,-1,2,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,4,3,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,-1,-1,-1,-1,-1,-1,-1,-1,5,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
939,-1,-1,-1,2,-1,-1,4,5,3,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
940,5,-1,-1,-1,-1,-1,4,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


## -1로 채워진 부분들을 채워주기 위해서 matrix factorization을 해봅시다
이때 torch를 활용해서 sgd기반의 방법으로 user embedding과 item embedding을 훈련시켜 보겠습니다.


In [15]:
# 위에 ml100k 클래스에서 torch dataloader를 불러오는 함수를 호출하여 데이터로더 형태의 train_loader, test_loader를 불러옵니다.
train_loader, test_loader=ml100k.load_torch_data()



이제 Matrix factorization model을 구현해봅시다. 

In [16]:
import torch
import torch.nn as nn

class MF(nn.Module):

    def __init__(self, args ,num_users,num_items) -> None:
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.latent_dim = args.latent_dim
        self.user_embedding = nn.Embedding(self.num_users, self.latent_dim)
        self.item_embedding = nn.Embedding(self.num_items, self.latent_dim)
        #self.dropout = nn.Dropout(p=args.dropout)


    def forward(self, x) -> None:
        # x: (batch_size, 2) : index 0:user, index 1:item
        user = x[:, 0]
        item = x[:, 1]
        user = self.user_embedding(user)
        #user shape: (batch_size, latent_dim) 우리가 원하는 유저가 표현된 latent_dim 차원의 벡터
        item = self.item_embedding(item) 
        #item shape: (batch_size, latent_dim) 우리가 원하는 아이템이 표현된 latent_dim 차원의 벡터

        out = torch.sum(user * item, dim=1) # 내적을 통해서 구하기,
        
        return out


여기서 잠깐 EMbedding이 어떻게 동작하는지 확인한번만 해봅시다

In [17]:
uemb_temp=nn.Embedding(ml100k.num_users, args.latent_dim)
iembe_temp=nn.Embedding(ml100k.num_items, args.latent_dim)

In [18]:
print(uemb_temp.weight.shape)
print(iembe_temp.weight.shape)

torch.Size([943, 10])
torch.Size([1682, 10])


In [19]:
uemb_temp.weight

Parameter containing:
tensor([[-1.4641,  0.3351, -1.2466,  ..., -0.9561, -0.4217,  1.0331],
        [-0.3160, -0.6751, -0.5141,  ...,  1.3306,  1.2104,  1.7352],
        [ 0.5514, -1.9808, -1.9849,  ..., -0.0810,  1.1391, -1.6611],
        ...,
        [ 1.2441,  0.0638,  1.0465,  ...,  0.9198, -1.2126,  0.1097],
        [-2.1287,  0.5005, -0.3823,  ...,  1.4006, -0.8987, -1.6914],
        [ 1.1890, -0.2972,  0.1160,  ...,  0.1206, -0.1253,  1.3920]],
       requires_grad=True)

In [20]:
iembe_temp.weight

Parameter containing:
tensor([[-0.4601,  0.8592, -0.6129,  ..., -1.5253, -0.6080, -1.2354],
        [ 0.3708,  0.6575, -0.1598,  ..., -1.2372, -0.0279,  0.5040],
        [-0.3926, -0.4646,  1.0980,  ..., -0.2687,  1.7898,  0.6122],
        ...,
        [-1.5988, -0.3890,  1.4416,  ...,  0.0285,  1.8501, -0.4362],
        [ 0.4506, -1.1946, -0.8430,  ..., -0.5151,  0.2456,  0.0662],
        [ 0.9731,  0.5596, -0.3561,  ..., -0.2391, -0.3338, -0.2886]],
       requires_grad=True)

즉 nn.Embedding을 통해서 유저, 아이템에 대한 embedding matrix를 저장하고 훈련시킬 수 잇는 네트워크라고 보시면 됩니다.(자연어에서 사용되는 nn.embedding과 같습니다.`)

그럼 이제 모델을 훈련시켜봅시다. 엄밀히 훈련시키려면 regularization,earlystopping등의 장치가 추가적으로 필요하지만, 돌아가는거만 확인하기 위해서 최대한 간단한 형태로 구현했습니다.

In [21]:
import torch
class Trainer():

    def __init__(self, model, optimizer, criterion, device):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device

    
    def train(self, data, test_data, epochs):
        for epoch in range(epochs):
            for (x,y) in data:
                x = x.to(self.device)
                y = y.to(self.device)
                self.optimizer.zero_grad()
                output = self.model(x)
                loss = self.criterion(output, y)
                loss.backward()
                self.optimizer.step()
                # print test loss
            test_loss = 0
            with torch.no_grad():
                for (x,y) in test_data:
                    x = x.to(self.device)
                    y = y.to(self.device)
                    output = self.model(x)
                    test_loss += self.criterion(output, y)

                #normalize test loss
            test_loss /= len(test_data)
            print(f"Epoch {epoch} train loss: {loss.item()} test loss: {test_loss.item()}")

        print("Training finished")




In [23]:
# model, optimizer, criterion, device를 정의합니다.
model = MF(args, ml100k.num_users, ml100k.num_items)
optimizer = torch.optim.Adam(model.parameters(),lr=0.05)
criterion = nn.MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
criterion.to(device)

# trainer를 정의합니다.
trainer = Trainer(model, optimizer, criterion, device)
trainer.train(train_loader, test_loader, 10)

cuda
Epoch 0 train loss: 3.3060355186462402 test loss: 2.6313536167144775
Epoch 1 train loss: 1.0687007904052734 test loss: 1.3431037664413452
Epoch 2 train loss: 1.0125938653945923 test loss: 1.250959038734436
Epoch 3 train loss: 1.0015122890472412 test loss: 1.226389765739441
Epoch 4 train loss: 0.8630713820457458 test loss: 1.222793698310852
Epoch 5 train loss: 0.9900162220001221 test loss: 1.2172927856445312
Epoch 6 train loss: 0.8205846548080444 test loss: 1.2488025426864624
Epoch 7 train loss: 0.8373615741729736 test loss: 1.2248910665512085
Epoch 8 train loss: 0.8997617363929749 test loss: 1.2664873600006104
Epoch 9 train loss: 0.9109413623809814 test loss: 1.2566653490066528
Training finished


이제 아까 빈칸이었던 user-item matrix를 채워봅시다!




In [24]:
user_item_matrix=dataframe.pivot(index='user', columns='item', values='rating').fillna(-1) #pandas로 하면 너무느려져서 numpy로 바구고 진행합시다
user_item_matrix=user_item_matrix.to_numpy()

In [25]:
model=model.cpu() #모델을 cpu로 바꿔주고
# 훈련된 embedding을 numpy로 바꿔줍니다.
np_userembedding=model.user_embedding.weight.detach().numpy() 
np_itemembedding=model.item_embedding.weight.detach().numpy()

In [26]:
user_item_matrix

array([[ 5.,  3.,  4., ..., -1., -1., -1.],
       [ 4., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [ 5., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1.,  5., -1., ..., -1., -1., -1.]])

## Q2. 아래 코드를 완성해주세요!

In [27]:
# 아래 코드를 완성해주세요
for i in range(ml100k.num_users):
    for j in range(ml100k.num_items):
        if user_item_matrix[i,j]==-1:
            user_item_matrix[i,j]= np.dot(np_userembedding[i],np_itemembedding[j])  # 물음표를 채워주세요! , HInt  : 내적 

## 결과

In [28]:
user_item_matrix

array([[ 5.00000000e+00,  3.00000000e+00,  4.00000000e+00, ...,
        -4.09730375e-01,  1.59113574e+00, -3.68152523e+00],
       [ 4.00000000e+00,  3.11689663e+00,  2.87994027e+00, ...,
         1.17439032e-03,  2.50048184e+00, -6.50377750e+00],
       [ 3.35102296e+00,  2.32496309e+00,  1.18042397e+00, ...,
        -4.65569645e-02,  1.84650517e+00, -3.89335322e+00],
       ...,
       [ 5.00000000e+00,  3.39868021e+00,  3.03889227e+00, ...,
         1.04722691e+00,  1.56511140e+00, -4.15220976e+00],
       [ 5.00220537e+00,  4.24875593e+00,  3.23860002e+00, ...,
        -3.69506091e-01,  3.66063595e+00, -3.79572558e+00],
       [ 5.05492067e+00,  5.00000000e+00,  4.14451599e+00, ...,
        -3.20231289e-01,  3.44614840e+00, -4.00939226e+00]])

### 추가로....
결과를 보면 범위가 -3 에서  +5보다 큰 값도 나왔는데, 일반적으로는 0.5~5 사이로 normalization을 시켜주는 것 같습니다. 이건 생략하겠습니다 ㅎㅎ..